In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib widget

layout = widgets.Layout(align_items = 'center')

<h1>REGRESIÓN LINEAL</h1>
<h2>SET DE DATOS</h2>  

In [ ]:
dfHelados = pd.read_csv('helados.csv')
dfHelados.head(5)

Se utilizarán las columnas correspondientes al ingreso de las familias, el precio del helado y la temperatura del día para intentar <strong>predecir la cantidad de helado consumido</strong>.

<h2>ECUACIÓN NORMAL</h2>                                                         
<ol>
    <li>Conformación de la matriz X a partir de las variables de entrada:
         $$
           \begin{bmatrix}
            1\quad x_1^0\quad x_2^0\quad x_3^0\\
            1\quad x_1^1\quad x_2^1\quad x_3^1\\
            ... \\
            1\quad x_1^m\quad x_2^m\quad x_3^m
           \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
Xtest = dfHelados.loc[:,'income':'temp'].values
x0 = np.ones(shape = (Xtest.shape[0],1))
X = np.hstack((x0, Xtest))

<ol start="2">
    <li>Conformación del vector de valores de salida:
        $$
        \begin{bmatrix}
            y^0\\
            y^1\\
            ... \\
            y^m
        \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
Y = dfHelados.loc[:,'cons'].values

<ol start="3">
    <li>Generación de la matriz transpuesta de X:
    </li>
</ol>

In [ ]:
Xtrans = X.transpose()

<ol start="4">
    <li>Aplicación de la fórmula de ecuación normal:
        $$
            \theta = \left(X^T X\right)^{-1}\left(X^T y\right)
        $$
    </li>
</ol>

In [ ]:
theta = np.linalg.inv(Xtrans.dot(X)).dot(Xtrans).dot(Y)
theta

<ol start="5">
    <li>Cálculo de los valores de salida predichos:
        $$
            h_{\theta}(x)=\theta_0x_0+\theta_1x_1+\theta_2x_2+\theta_3x_3
        $$
    </li>
</ol>

In [ ]:
h = X.dot(theta)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4),tight_layout = True)
ax.scatter(np.arange(0,30),h,marker='x',c = np.linspace(0,1,30), label='Consumo predicho',alpha = 0.9)
ax.scatter(np.arange(0,30),Y,marker='o',c = np.linspace(0,1,30), label='Consumo real',alpha = 0.9)
ax.grid(True)
ax.legend();

<h2>GRADIENTE DESCENDENTE</h2>              

<ol>
    <li>Normalización y escalamiento de las variables de entrada:
         $$
            x_i := \frac{x_i-\mu_i}{\sigma}
         $$
    </li>
</ol>

In [ ]:
col_mask = ['income', 'price', 'temp']
dfHelados2 = dfHelados.loc[:, col_mask]
dfH_norm = (dfHelados2 - dfHelados2.mean(axis = 0)) / dfHelados2.std(axis = 0)  
dfH_norm.head(5)

<ol start="2">
    <li>Conformación de la matriz X a partir de las variables de entrada normalizadas:
         $$
           \begin{bmatrix}
            1\quad x_1^0\quad x_2^0\quad x_3^0\\
            1\quad x_1^1\quad x_2^1\quad x_3^1\\
            ... \\
            1\quad x_1^m\quad x_2^m\quad x_3^m
           \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
X_gd = dfH_norm.values
x0_gd = np.ones(shape = (X_gd.shape[0],1))
X_gd = np.hstack((x0_gd, X_gd))

<ol start="3">
    <li>Conformación del vector de valores de salida:
        $$
        \begin{bmatrix}
            y^0\\
            y^1\\
            ... \\
            y^m
        \end{bmatrix}
        $$
    </li>
</ol>

In [ ]:
Y_gd = dfHelados.loc[:,'cons'].values.reshape(-1, 1)

<ol start="4">
    <li>Selección del parámetro $\alpha$
    </li>
</ol>

In [ ]:
alpha_slider = widgets.FloatSlider(min = 0,max = 0.9,step = 0.001,value = 0.5,description = r'$\alpha$')

<ol start="5">
    <li>Aplicación del algoritmo iterativo de gradiente descendente:
        $$
        \theta_j := \theta_j - \alpha \frac{\partial}{\partial\theta_j}J(\theta_0, \theta_1)
        $$
    </li>
</ol>

In [ ]:
def plot_J_gd(alpha,X,Y):
    m = X.shape[0]
    
    t = np.zeros(shape = (X.shape[1],1))
     
    J = np.array([])

    for i in range(0,10):
        h_gd = X.dot(t[:,-1])
        h_gd = h_gd.reshape(-1, 1)
        J = np.append(J, 1/(2 * m) * np.sum(np.square(h_gd - Y)))
        t_temp = np.array([]) 
        for j in range(0,t.shape[0]):
            t_temp = np.append(t_temp, t[j,i] - alpha/m * (X[:,j].reshape(1, -1).dot(h_gd - Y)))
        t = np.hstack((t,t_temp.reshape(-1, 1)))
    return t, J

<ol start="6">
    <li>Cálculo de los valores de salida predichos:
        $$
            h_{\theta}(x)=\theta_0x_0+\theta_1x_1+\theta_2x_2+\theta_3x_3
        $$
    </li>
</ol>

In [ ]:
def h_gd_value(t,X):
    h = X.dot(t[:,-1])
    return h

In [ ]:
plots_gd = widgets.Output()

with plots_gd:
    fig_gd, axs_gd = plt.subplots(3,1,figsize = (7,8),tight_layout = True)
    fig_gd.suptitle(r'Gradiente descendente')
    
    axs_gd[1].set_title(fr'Variación de función costo $J(\theta)$')
    t_gd, J_gd = plot_J_gd(alpha_slider.value,X_gd,Y_gd)
    linej, = axs_gd[1].plot(J_gd)
    axs_gd[1].plot()
    axs_gd[1].set_ylabel(r'$J(\theta)$')   
    axs_gd[1].set_xlabel(r'N° de iteraciones')   
    axs_gd[1].grid(True)
    
    axs_gd[0].set_title(fr'Hipótesis vs Valores de salida reales')
    h_gd = h_gd_value(t_gd,X_gd)
    lineh, = axs_gd[0].plot(h_gd,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gd[0].scatter(np.arange(0,30),Y_gd,marker='o',label='Consumo real',alpha = 0.9)
    axs_gd[0].legend()
    axs_gd[0].grid(True)
                        
    axs_gd[2].set_title(fr'Variación de $\theta_j$')
    theta0, = axs_gd[2].plot(t_gd[0,:],label=r'$\theta_0$')
    theta1, = axs_gd[2].plot(t_gd[1,:],label=r'$\theta_1$')
    theta2, = axs_gd[2].plot(t_gd[2,:],label=r'$\theta_2$')
    theta3, = axs_gd[2].plot(t_gd[3,:],label=r'$\theta_3$')
    axs_gd[2].set_ylabel(r'$\theta$')   
    axs_gd[2].set_xlabel(r'N° de iteraciones')   
    axs_gd[2].grid(True)
    axs_gd[2].legend()

In [ ]:
out_t0_gd = widgets.HTMLMath(value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_gd[0,-1]:.4f}')
    
out_t1_gd = widgets.HTMLMath(value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_gd[1,-1]:.4f}')

out_t2_gd = widgets.HTMLMath(value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_gd[2,-1]:.4f}')
    
out_t3_gd = widgets.HTMLMath(value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_gd[3,-1]:.4f}')


In [ ]:
def update_gd (change):
    t_gd, J_gd = plot_J_gd(alpha_slider.value,X_gd,Y_gd)
    linej.set_ydata(J_gd)
    h_gd = h_gd_value(t_gd,X_gd)
    lineh.set_ydata(h_gd)
    theta0.set_ydata(t_gd[0,:])
    theta1.set_ydata(t_gd[1,:])
    theta2.set_ydata(t_gd[2,:])
    theta3.set_ydata(t_gd[3,:])
    out_t0_gd.value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_gd[0,-1]:.4f}'
    out_t1_gd.value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_gd[1,-1]:.4f}'
    out_t2_gd.value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_gd[2,-1]:.4f}'
    out_t3_gd.value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_gd[3,-1]:.4f}'
    
alpha_slider.observe(update_gd ,'value')

widgets.VBox([plots_gd,alpha_slider, out_t0_gd, out_t1_gd, out_t2_gd, out_t3_gd],layout=layout)



<div class="alert alert-block alert-success">
<b>Conclusiones:</b>
    <ol>
    <li>Si $\alpha$ es muy pequeño, la convergencia del gradiente descendente es lenta (requiere más iteraciones).</li>
    <li>El valor de la función de costo $J(\theta)$ decrece en cada iteración.</li>
    <li>Mientras menor sea el valor de la función costo $J(\theta)$, la función hipótesis realizará mejores predicciones de los valores de salida $y$.</li>
</ol>
    
</div>

<h3>Gradiente descendente sin normalizar</h3>        

In [ ]:
dfHelados2.head(5)

In [ ]:
X_gdSn = dfHelados2.values
x0_gdSn = np.ones(shape = (X_gdSn.shape[0],1))
X_gdSn = np.hstack((x0_gdSn, X_gdSn))

Y_gdSn = dfHelados.loc[:,'cons'].values.reshape(-1, 1)

In [ ]:
alphaSn_slider = widgets.FloatSlider(min = 0,max = 0.09,step = 0.001,value = 0.05,description = r'$\alpha$')

In [ ]:
plots_gdSn = widgets.Output()

with plots_gdSn:
    fig_gdSn, axs_gdSn = plt.subplots(3,1,figsize = (7,8),tight_layout = True)
    fig_gdSn.suptitle(r'Gradiente descendente sin normalización')
    
    axs_gdSn[1].set_title(fr'Variación de función costo $J(\theta)$')
    t_gdSn, J_gdSn = plot_J_gd(alphaSn_slider.value,X_gdSn,Y_gdSn)
    linejSn, = axs_gdSn[1].plot(J_gdSn)
    axs_gdSn[1].set_ylabel(r'$J(\theta)$')   
    axs_gdSn[1].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[1].grid(True)
    
    axs_gdSn[0].set_title(fr'Hipótesis vs Valores de salida reales')
    h_gdSn = h_gd_value(t_gdSn,X_gdSn)
    linehSn, = axs_gdSn[0].plot(h_gdSn,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdSn[0].scatter(np.arange(0,30),Y_gdSn,marker='o',label='Consumo real',alpha = 0.9)
    axs_gdSn[0].legend()
    axs_gdSn[0].grid(True)
    
    axs_gdSn[2].set_title(fr'Variación de $\theta_j$')
    theta0Sn, = axs_gdSn[2].plot(t_gdSn[0,:],label=r'$\theta_0$')
    theta1Sn, = axs_gdSn[2].plot(t_gdSn[1,:],label=r'$\theta_1$')
    theta2Sn, = axs_gdSn[2].plot(t_gdSn[2,:],label=r'$\theta_2$')
    theta3Sn, = axs_gdSn[2].plot(t_gdSn[3,:],label=r'$\theta_3$')
    axs_gdSn[2].set_ylabel(r'$\theta$')   
    axs_gdSn[2].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[2].grid(True)
    axs_gdSn[2].legend()
    

In [ ]:
out_t0_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_gdSn[0,-1]:.4e}')
    
out_t1_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_gdSn[1,-1]:.4e}')

out_t2_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_gdSn[2,-1]:.4e}')
    
out_t3_gdSn = widgets.HTMLMath(value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_gdSn[3,-1]:.4e}')

In [ ]:
def update_gdSn (change):
    t_gdSn, J_gdSn = plot_J_gd(alphaSn_slider.value,X_gdSn,Y_gdSn)
    axs_gdSn[1].cla()
    linejSn, = axs_gdSn[1].plot(J_gdSn)
    axs_gdSn[1].set_ylabel(r'$J(\theta)$')   
    axs_gdSn[1].set_xlabel(r'N° de iteraciones')  
    axs_gdSn[1].grid(True)
    
    h_gdSn = h_gd_value(t_gdSn,X_gdSn)
    axs_gdSn[0].cla()
    linehSn, = axs_gdSn[0].plot(h_gdSn,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdSn[0].scatter(np.arange(0,30),Y_gdSn,marker='o',label='Consumo real',alpha = 0.9)
    axs_gdSn[0].legend()
    axs_gdSn[0].grid(True)
    
    axs_gdSn[2].cla()
    theta0Sn, = axs_gdSn[2].plot(t_gdSn[0,:],label=r'$\theta_0$')
    theta1Sn, = axs_gdSn[2].plot(t_gdSn[1,:],label=r'$\theta_1$')
    theta2Sn, = axs_gdSn[2].plot(t_gdSn[2,:],label=r'$\theta_2$')
    theta3Sn, = axs_gdSn[2].plot(t_gdSn[3,:],label=r'$\theta_3$')
    axs_gdSn[2].set_ylabel(r'$\theta$')   
    axs_gdSn[2].set_xlabel(r'N° de iteraciones')   
    axs_gdSn[2].grid(True)
    axs_gdSn[2].legend();

    out_t0_gdSn.value = fr'El valor de $\theta_0$ después de 10 iteraciones es: {t_gdSn[0,-1]:.4e}'
    out_t1_gdSn.value = fr'El valor de $\theta_1$ después de 10 iteraciones es: {t_gdSn[1,-1]:.4e}'
    out_t2_gdSn.value = fr'El valor de $\theta_2$ después de 10 iteraciones es: {t_gdSn[2,-1]:.4e}'
    out_t3_gdSn.value = fr'El valor de $\theta_3$ después de 10 iteraciones es: {t_gdSn[3,-1]:.4e}'
    
alphaSn_slider.observe(update_gdSn ,'value')

widgets.VBox([plots_gdSn,alphaSn_slider, out_t0_gdSn, out_t1_gdSn, out_t2_gdSn, out_t3_gdSn],layout=layout)


<div class="alert alert-block alert-danger">
<b>Conclusiones:</b> 
    <ol>
        <li>Si las variables de entrada (features) $x_j$ poseen rangos de valores muy diferentes, 
            es <b>necesario</b> realizar la normalización de las mismas para asegurar la 
            convergencia del gradiente descendente.</li>
    </ol>
</div>

<h3>Gradiente descendente con regularización</h3>    

In [ ]:
df_util = dfH_norm.loc[:,['income','price','temp']]

In [ ]:
extra1 = df_util['income'].values * 2 + np.random.randn(df_util.shape[0]) 
df_extra1 = pd.DataFrame(extra1)
df_extra1 = (df_extra1 - df_extra1.mean(axis = 0)) / df_extra1.std(axis = 0)  


In [ ]:
dfH_reg = pd.concat([df_util, df_extra1],axis=1)
dfH_reg.head(5)

In [ ]:
dfH_reg.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
X_reg = dfH_reg.values
x0_reg = np.ones(shape = (X_reg.shape[0],1))
X_reg = np.hstack((x0_reg, X_reg))

In [ ]:
Y_reg = dfHelados.loc[:,'cons'].values.reshape(-1, 1)

In [ ]:
alphaReg_slider = widgets.FloatSlider(min = 0,max = 0.9,step = 0.01,value = 0.5,description = r'$\alpha$')
lambdaReg_slider = widgets.FloatSlider(min = 0,max = 10.00,step = 0.1,value = 0.00,description = r'$\lambda$')

In [ ]:
def plot_J_gdReg(alpha, X, Y, l):
    m = X.shape[0]
    
    t = np.zeros(shape = (X.shape[1],1))
     
    J = np.array([])

    for i in range(0,20):
        h_gd = X.dot(t[:,-1])
        h_gd = h_gd.reshape(-1, 1)
        
        J = np.append(J, 1/(2 * m) *(np.sum(np.square(h_gd - Y)) + l * sum(np.square(t[1:,i]))))
            
        t_temp = np.array([]) 
        
        for j in range(0,t.shape[0]):
            if j == 0:
                t_temp = np.append(t_temp, t[j,i] - alpha/m * (X[:,j].reshape(1, -1).dot(h_gd - Y)))
            else:
                t_temp = np.append(t_temp, t[j,i] - alpha/m * ((X[:,j].reshape(1, -1).dot(h_gd - Y)) + l * t[j,i]))
                
        t = np.hstack((t,t_temp.reshape(-1, 1)))
    return t, J

In [ ]:
plots_gdReg = widgets.Output()

with plots_gdReg:
    fig_gdReg, axs_gdReg = plt.subplots(2,1,figsize = (7,6),tight_layout = True)
    fig_gdReg.suptitle(r'Gradiente descendente con regularización')
    
    axs_gdReg[1].set_title(r'Variación de función costo $J(\theta)$')
    t_gdReg, J_gdReg = plot_J_gdReg(alphaReg_slider.value,X_reg,Y_reg,lambdaReg_slider.value)
    linejReg, = axs_gdReg[1].plot(J_gdReg)
    axs_gdReg[1].set_ylabel(r'$J(\theta)$')   
    axs_gdReg[1].set_xlabel(r'N° de iteraciones')   
    axs_gdReg[1].grid(True)
    
    axs_gdReg[0].set_title(r'Hipótesis vs Valores de salida reales')
    h_gdReg = h_gd_value(t_gdReg,X_reg)
    linehReg, = axs_gdReg[0].plot(h_gdReg,'rx', label='Consumo predicho',alpha = 0.9)
    axs_gdReg[0].plot(Y_reg,'o',label='Consumo real',alpha = 0.9)
    axs_gdReg[0].legend()
    axs_gdReg[0].grid(True)
    

In [ ]:
out_t1_gdReg = widgets.HTMLMath(value = fr'El valor de $\theta_1$ después de 20 iteraciones es: {t_gdReg[1,-1]:.4f}')
out_t4_gdReg = widgets.HTMLMath(value = fr'El valor de $\theta_4$ después de 20 iteraciones es: {t_gdReg[4,-1]:.4f}')

out_J = widgets.HTMLMath(value = fr'El valor de $J(\theta)$  después de 20 iteraciones es: {J_gdReg[-1]:.5f}')

In [ ]:
def update_gdReg (change):
    t_gdReg, J_gdReg = plot_J_gdReg(alphaReg_slider.value,X_reg,Y_reg,lambdaReg_slider.value)
    linejReg.set_ydata(J_gdReg)
    h_gdReg = h_gd_value(t_gdReg,X_reg)
    linehReg.set_ydata(h_gdReg)
    
    out_t1_gdReg.value = fr'El valor de $\theta_1$ después de 20 iteraciones es: {t_gdReg[1,-1]:.4f}'
    out_t4_gdReg.value = fr'El valor de $\theta_4$ después de 20 iteraciones es: {t_gdReg[4,-1]:.4f}'
    
    out_J.value = fr'El valor de $J(\theta)$ después de 20 iteraciones es: {J_gdReg[-1]:.5f}'
    
alphaReg_slider.observe(update_gdReg ,'value')
lambdaReg_slider.observe(update_gdReg ,'value')

widgets.VBox([plots_gdReg,alphaReg_slider,\
              lambdaReg_slider, \
              out_t1_gdReg, \
              out_t4_gdReg, \
              out_J], layout=layout)


<div  class="alert alert-block alert-warning">
<b>Conclusiones:</b> 
    <ol>
        <li>Cuando existe una fuerte correlación entre 2 variables de entrada, es necesario aplicar regularización 
            para simplificar el modelo.</li>
        <li>Una variable correlacionada a otra no tendrá un gran impacto en mejorar el modelo, solo le 
            agregará complejidad.</li>
        <li>Aplicar regularización evita problemas de overfitting.</li>
        <li>El parámetro $\lambda$ es un número real positivo que controla el impacto de la regularización.</li>
        <li>Al aumentar el grado de regularización, aumenta el valor de la función de costo $J(\theta)$.</li>
    </ol>
</div>

<h2>IMPLEMENTACIÓN CON LIBRERÍAS ESPECIALIZADAS</h2>

<ol>
    <li>Se utiliza la librería <strong>scikit-learn</strong>
    </li>
</ol>

In [ ]:
from sklearn.linear_model import LinearRegression

<ol start="2">
    <li>Los arreglos de entrada y salida deben ser <strong>NumPy arrays</strong>.
        <ul>
            <li>El arreglo de <strong>X</strong> debe tener <strong>2 dimensiones</strong>: cada columna debe corresponder con una variable de entrada y cada fila con una observación particular.</li>
            <li>El arreglo de <strong>Y</strong> debe tener <strong>1 dimensión</strong>, siendo cada elemento la salida de cada una de las observaciones</li>
        </ul>
    </li>
</ol>

In [ ]:
Xtest = dfHelados.loc[:,'income':'temp'].values
Xtest

In [ ]:
Y = dfHelados.loc[:,'cons'].values
Y

<ol start="3">
    <li>Se normalizan los valores de los parámetros de entrada. 
        Para ello se utiliza la clase <strong>StandardScaler</strong>.
        Algunos de los parámetros que posee esta clase son:
        <ul>
          <li><strong>with_mean</strong> decide si centra los 
              datos antes de realizar el escalado.</li>
          <li><strong>with_std</strong> decide si escala los 
              datos para tener varianza unitaria, es decir, desviación unitaria.</li>
        </ul>
        Por defecto with_mean y with_std se consideran True.
    <br> 
    Algunos de sus métodos son:
         <ul>
             <li><strong>fit</strong> calcula la media y desviación estándar 
                 que será usado luego para realizar el escalado.</li>
             <li><strong>transform</strong> realiza la estandarización de los
                 datos mediante el centrado y escalado.</li>
             <li><strong>fit_transform</strong> calcula la media y desviación 
                 estándar de los datos y luego realiza su transformación.</li>
         </ul>
    </li>
</ol>

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xtest = scaler.fit_transform(Xtest)
Xtest

<ol style="list-style-type:none;">
    <li>
        Es posible acceder a los atributos de la clase StandardScaler entrenada:
    </li>
</ol>

In [ ]:
print(f"""Los valores de media son: {scaler.mean_}
Los valores de desviación estándar son: {scaler.var_}""")

<ol start="4">
    <li>Se crea una instancia de la clase <strong>LinearRegression</strong>, la cual tiene muchos parámetros opcionales:
        <ul>
          <li><strong>fit_intercept</strong> decide si se calcula el término independiente (True) o si se considera 0 (False).</li>
          <li><strong>copy_X</strong> decide si se copian (True) o se sobreescriben las variables de entrada (False).</li>
        </ul>
        Por defecto fit_intercept y copy_X se consideran True, mientras que normalize se considera False.
    </li>
</ol>

In [ ]:
model = LinearRegression()

<ol start="5">
    <li>Se entrena el modelo, lo cual determina los valores de los coeficientes que corresponden al mejor valor de la función costo
    </li>
</ol>

In [ ]:
model.fit(Xtest, Y)

<ol start="6">
    <li>Se pueden obtener los atributos del modelo:
        <ul>
            <li> Los valores de los coeficientes del modelo entrenado.
            </li>
        </ul>
    </li>
</ol>

In [ ]:
print(model.intercept_,model.coef_)

<ol start="7">
    <li>Se calculan los valores predichos de salida haciendo uso del modelo entrenado.
    </li>
</ol>

In [ ]:
predictions = model.predict(Xtest)
predictions

In [ ]:
skl = widgets.Output()

with skl:
    fig_p, ax_p = plt.subplots(figsize = (6,4),tight_layout = True)
    fig_p.suptitle(r'Hipótesis vs Valores de salida reales')
    ax_p.plot(np.arange(0,len(Y)),predictions,'b+',markersize=8,label='Predicción con sklearn',alpha = 0.9)
    ax_p.scatter(np.arange(0,len(Y)),Y,marker='o',label='Salida real',alpha = 0.9,facecolors='none', edgecolors='g')
    lineh_skl, = ax_p.plot(np.arange(0,len(Y)),h_gd,'rx', markersize=8,label='Predicción',alpha = 0.9)
    ax_p.grid(True)
    ax_p.legend();

def update_skl (change):
    t_gd, J_gd = plot_J_gd(alpha_slider.value,X_gd,Y_gd)
    h_gd = h_gd_value(t_gd,X_gd)
    lineh_skl.set_ydata(h_gd)

alpha_slider.observe(update_skl ,'value')

widgets.VBox([skl,alpha_slider],layout = layout)



<ol start="7">
    <li>Se calcula un puntaje para el modelo, el mismo equivale al porcentaje de predicciones correctas realizadas por el modelo.
    </li>
</ol>

In [ ]:
model.score(Xtest,Y)